In [1]:
import segyio
import numpy as np

In [2]:
def rider(file_path):
   with segyio.open(file_path, 'r') as segyfile:
       y = [i for i in segyfile.xlines]
       x = [i for i in segyfile.ilines]
       z = [i for i in segyfile.samples]
       cubic = segyio.tools.cube(segyfile)
       tras = {}
       for i in range(len(x)):
           for j in range(len(y)):
               tras[str(x[i]) + "_" + str(y[j])] = cubic[i][j][:]
   return x, y, z, tras


In [3]:
def assemble_cube(x, y, z, tras):
   cube = np.zeros((len(x), len(y), len(z)))
   for i, xi in enumerate(x):
       for j, yj in enumerate(y):
           key = f"{xi}_{yj}"
           if key in tras:
               cube[i][j][:] = tras[key]
   return cube


In [4]:
def read_segy(file_path):
   with segyio.open(file_path, "r") as segyfile:
       segyfile.mmap()
       return segyio.tools.cube(segyfile)

In [5]:
def calculate_pearson_coefficient(cube1, cube2):
   return np.corrcoef(cube1.flatten(), cube2.flatten())[0, 1]

In [6]:
X1, Y1, Z1, T1 = rider('C:/HV/Seismic/1.3_comparison_of_cubes/Volume_attribute1_RMS_TWT.segy')
X2, Y2, Z2, T2 = rider('C:/HV/Seismic/1.3_comparison_of_cubes/Volume_attribute2_Amplitude_contrast_TWT.segy')
cube1 = assemble_cube(X1, Y1, Z1, T1)
cube2 = assemble_cube(X2, Y2, Z2, T2)


In [8]:
# Рассчитываем коэффициенты Пирсона
pearson_coefficient = calculate_pearson_coefficient(cube1, cube2)
print(pearson_coefficient)

# Создаем confusion matrix
confusion_matrix = np.array([[1, pearson_coefficient], [pearson_coefficient, 1]])
print(confusion_matrix)


-0.05365608202155685
[[ 1.         -0.05365608]
 [-0.05365608  1.        ]]
